# Sentimentanalyse von Plenarprotokollen mit Python

Dieses Dokument dient zum Erkunden der Datensätze, die mithilfe der in diesem Ordner verfügbaren Skriptdateien ``process_pp.py`` und ``dictionary_analysis.py`` erstellt wurden. Der größte Teil der Rechenarbeit wurde also mithilfe dieser Skripte bereits durchgeführt. Das Laden des aufbereiteten Datensatzes kann trotzdem einige Minuten dauern.

---
## Vorbereitung

Zunächst sollen alle im Folgenden benötigte Python-Pakete geladen werden. Mit dem Code in der nächsten Zelle kann überprüft werden, ob diese Pakete installiert sind. Zusätzlich wird ein deutsches Sprachmodell ``de_core_news_sm`` für spaCy heruntergeladen.

In [ ]:
!pip install spacy pandas seaborn matplotlib
!python -m spacy download de_core_news_sm

Als nächstes werden die installierten Pakete für das aktuelle Skript importiert und das Sprachmodell verfügbar gemacht.

In [ ]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("de_core_news_sm")
import pandas as pd
from glob import glob
import operator
import seaborn
import re
import matplotlib.pyplot as plt

Anschließend werden die für die Analyse nötigen Daten eingelesen. Dieser Prozess kann einige Zeit und Speicher in Anspruch nehmen.

In [ ]:
dateien = glob("plenarprotokolle/pp19/*.xml.spacy.sentiment")
spacy_db = {}
for protokoll in dateien:
    protokoll_daten = DocBin(store_user_data=True).from_bytes(open(protokoll, "rb").read())
    protokoll_daten = list(protokoll_daten.get_docs(nlp.vocab))
    spacy_db[protokoll] = protokoll_daten

---
## Analyse

### Grundlegende Statistiken

Nun sind alle für die Analyse nötigen Ressourcen verfügbar. Zum Einen kann jetzt natürlich mit diesen experimentiert werden, zum Anderen bietet der Rest dieses Dokuments eine Führung durch beispielhafte Analysevorgänge. Zum Beispiel können grundlegende Statistiken zum Datensatz zusammengestellt werden, wie dessen allgemeiner Umfang:

In [ ]:
print("Der Datensatz enthält", len(spacy_db), "Protokolle.")

anzahl_reden = 0
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        anzahl_reden += 1
print("Diese Protokolle enthalten insgesamt", anzahl_reden, "Reden.")

anzahl_token = 0
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        anzahl_token += len(rede)
print("Diese Reden enthalten insgesamt", anzahl_token, "Token.")

Eine andere Möglichkeit ist ein Überblick über die Verteilung der Reden und Redner auf Parteien. Hierzu sollten wir aus den Daten zunächst eine entsprechende Sammlung zusammenstellen:

In [ ]:
redner_parteien = {}
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        partei = rede.user_data["meta"]["redner_partei"]
        redner = rede.user_data["meta"]["redner_name"]
        if not partei in redner_parteien.keys():
            redner_parteien[partei] = [redner]
        else:
            redner_parteien[partei].append(redner)

In [ ]:
for partei, namen in redner_parteien.items():
    print("Abgeordnete der Partei", partei, "sprachen", len(namen), "mal (" + "%.2f" % round((len(namen)/anzahl_reden)*100, 2), "%).")

In [ ]:
redneranzahl_parteien = {}
for partei,namen in redner_parteien.items():
    redneranzahl_parteien[partei] = len(namen)
redneranzahl_parteien = pd.DataFrame(redneranzahl_parteien.items(), columns=["partei","anzahl_reden"]).set_index("partei")
redneranzahl_parteien.plot.bar()

---
Indem wir das Wörterbuch weiterverarbeiten, können wir sowohl herausfinden, wie viele Male die jeweiligen Redner sprachen, und wie viele Redner für jede Partei sprachen.

In [ ]:
redner_parteien_unik = {}
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        partei = rede.user_data["meta"]["redner_partei"]
        redner = rede.user_data["meta"]["redner_name"]
        if not partei in redner_parteien_unik.keys():
            redner_parteien_unik[partei] = {}
            redner_parteien_unik[partei][redner] = 1
        else:
            if not redner in redner_parteien_unik[partei]:
                redner_parteien_unik[partei][redner] = 1
            else:
                redner_parteien_unik[partei][redner] += 1

In [ ]:
for partei, namen in redner_parteien_unik.items():
    print("Für die Partei", partei, "sprachen", len(namen), "verschiedene Redner.")

In [ ]:
for partei, namen in redner_parteien_unik.items():
    top_redner = max(namen.items(), key=operator.itemgetter(1))
    print("Fleißigster Redner für die Partei", partei, "war", top_redner[0], "mit", top_redner[1], "Wortmeldungen")

---
### Sentimentanalyse

An dieser Stelle sollen einige Beispiele für tatsächliche Sentimentanalyse veranschaulicht werden, angefangen mit den durchschnittlichen Polaritätswerten nach Partei und dem positivsten bzw. negativsten Redner.

---
Zuerst sollen die Polaritätswerte nach Parteien veranschaulicht werden.

Für eine vernünftige Vergleichbarkeit werden zunächst die Polaritätswerte normalisiert, indem sie verhundertfacht und anschließend mit der Länge der Rede verrechnet werden. Reden, in denen keines der Token in SentiWS gefunden wurde, werden nicht in die Analyse miteinbezogen.

In [ ]:
partei_sentiws_werte = {}
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        partei = rede.user_data["meta"]["redner_partei"]
        sentiws_score = rede.user_data["sentiws"]["sentiment_score"]
        laenge = len(rede)
        if not laenge == 0 and not len(rede.user_data["sentiws"]) <= 1:
            score_relativ = round((sentiws_score/laenge)*100, 3)
            if not partei in partei_sentiws_werte.keys():
                partei_sentiws_werte[partei] = [score_relativ]
            else:
                partei_sentiws_werte[partei].append(score_relativ)

In [ ]:
for partei, sentiment_werte in partei_sentiws_werte.items():
    print("Reden von Abgeordneten der Partei", partei, "haben eine durchschnittliche Polarität von", round(sum(sentiment_werte)/len(sentiment_werte), 3), ", maximal", max(sentiment_werte), "und minimal", min(sentiment_werte), ".")

In [ ]:
partei_sentiws_werte_df = pd.DataFrame(partei_sentiws_werte.items(), columns=["partei","werte"])
partei_sentiws_werte_df = partei_sentiws_werte_df.explode("werte")
partei_sentiws_werte_df["werte"] = pd.to_numeric(partei_sentiws_werte_df["werte"])

In [ ]:
partei_sentiws_werte_plot = seaborn.violinplot(y="werte", x="partei", data=partei_sentiws_werte_df, saturation=0.8)
partei_sentiws_werte_plot.set_xticklabels(["Unbekannt","AfD","CDU","FDP","LINKE","GRÜNE","SPD","CSU","Parteilos"])
partei_sentiws_werte_plot.set(xlabel="", ylabel="Durchschn. Polarität")

---
Das gleiche lässt sich jedoch auch ohne Normalisierung der Polaritätswerte durchführen.

In [ ]:
partei_sentiws_werte = {}
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        partei = rede.user_data["meta"]["redner_partei"]
        sentiws_score = rede.user_data["sentiws"]["sentiment_score"]
        laenge = len(rede)
        if not laenge == 0 and not len(rede.user_data["sentiws"]) <= 1:
            if not partei in partei_sentiws_werte.keys():
                partei_sentiws_werte[partei] = [sentiws_score]
            else:
                partei_sentiws_werte[partei].append(sentiws_score)

In [ ]:
partei_sentiws_werte_df = pd.DataFrame(partei_sentiws_werte.items(), columns=["partei","werte"])
partei_sentiws_werte_df = partei_sentiws_werte_df.explode("werte")
partei_sentiws_werte_df["werte"] = pd.to_numeric(partei_sentiws_werte_df["werte"])

In [ ]:
partei_sentiws_werte_plot = seaborn.violinplot(y="werte", x="partei", data=partei_sentiws_werte_df, saturation=0.8)
partei_sentiws_werte_plot.set_xticklabels(["Unbekannt","AfD","CDU","FDP","LINKE","GRÜNE","SPD","CSU","Parteilos"])
partei_sentiws_werte_plot.set(xlabel="", ylabel="Durchschn. Polarität")

---
### Analyse von Entitäten
Als nächstes wird ein Katalog aller im Korpus vorkommender Entitäten erstellt. Anschließend werden diesen die Polaritätswerte der Reden zugeordnet, in denen sie vorkommen.

In [ ]:
ents_sentiment = {}
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        ents = list(set(rede.user_data["entitaeten"]))
        sentiment = rede.user_data["sentiws"]["sentiment_score"]
        for ent in ents:
            ent = re.sub('[^A-Za-z0-9\- ,äöüÄÖÜß]+', '', ent)
            if not nlp.vocab[ent].is_stop and not nlp.vocab[ent.lower()].is_stop and len(ent) > 2:
                if not ent in ents_sentiment.keys():
                    ents_sentiment[ent] = [sentiment]
                else:
                    ents_sentiment[ent].append(sentiment)

An dieser Stelle sollen die mit den Entitäten verbundenen Reden auf mehrere Arten untersucht werden. Erstens können die durchschnittlichen Polaritätswerte für Reden gemessen werden, in denen die Entität vorkommt.

In [ ]:
ents_sentiment_avg = {ent: sum(sentiment)/len(sentiment) for ent, sentiment in ents_sentiment.items() if len(sentiment) >= 25}
ents_sentiment_avg_df = pd.DataFrame(ents_sentiment_avg.items(), columns=["ent","sentiment"]).set_index("ent")
ents_sentiment_avg_df.boxplot()

Hier sind z.B. die 10 Entitäten in den positivsten Reden:

In [ ]:
ents_sentiment_avg_df.sort_values("sentiment").tail(10)

Andererseits kann auch die Variabilität der Redensentimente, also negativste Rede minus positivste Rede, ermittelt werden.

In [ ]:
ents_sentiment_span = {ent: abs(min(sentiment)-max(sentiment)) for ent, sentiment in ents_sentiment.items() if len(sentiment) >= 5}
ents_sentiment_span_df = pd.DataFrame(ents_sentiment_span.items(), columns=["ent","sentiment"]).set_index("ent")
seaborn.violinplot(y="sentiment", data=ents_sentiment_span_df)

Oder die mit den Hauptthemen, also Entitäten, die in mehr als 50 Reden im Korpus vorkommen, verbundenen Sentimente.

In [ ]:
ents_sentiment_hauptthemen = {ent: sum(sentiment)/len(sentiment) for ent, sentiment in ents_sentiment.items() if len(sentiment) >= 50}
ents_sentiment_hauptthemen_df = pd.DataFrame(ents_sentiment_hauptthemen.items(), columns=["ent","sentiment"]).set_index("ent")
seaborn.violinplot(y="sentiment", data=ents_sentiment_hauptthemen_df)

---
### Polaritätswerte im Laufe der Zeit

Auch diachronisch können Polaritäten betrachtet werden: z.B. für Reden, die die Entitäten *Syrien* und *Europa* erwähnen, im Laufe der Wahlperiode.

In [ ]:
sitzung_sentiws_syrien = {}
sitzung_sentiws_europa = {}

for datei, protokoll in spacy_db.items():
    sitzungsnr = protokoll[0].user_data["meta"]["sitzungsnr"]
    for rede in protokoll:
        if "Syrien" in rede.user_data["entitaeten"]:
            sentiws_score = rede.user_data["sentiws"]["sentiment_score"]
            laenge = len(rede)
            if not laenge == 0 and not len(rede.user_data["sentiws"]) <= 1:
                if not sitzungsnr in sitzung_sentiws_syrien.keys():
                    sitzung_sentiws_syrien[sitzungsnr] = [sentiws_score]
                else:
                    sitzung_sentiws_syrien[sitzungsnr].append(sentiws_score)
        if "Europa" in rede.user_data["entitaeten"]:
            sentiws_score = rede.user_data["sentiws"]["sentiment_score"]
            laenge = len(rede)
            if not laenge == 0 and not len(rede.user_data["sentiws"]) <= 1:
                if not sitzungsnr in sitzung_sentiws_europa.keys():
                    sitzung_sentiws_europa[sitzungsnr] = [sentiws_score]
                else:
                    sitzung_sentiws_europa[sitzungsnr].append(sentiws_score)

Anschließend können diese in einen Datensatz zusammengeführt werden visualisiert werden.

In [ ]:
sitzung_sentiws_syrien_df = pd.DataFrame(sitzung_sentiws_syrien.items(), columns=["sitzung","sentiment_syrien"]).set_index("sitzung")
sitzung_sentiws_europa_df = pd.DataFrame(sitzung_sentiws_europa.items(), columns=["sitzung","sentiment_europa"]).set_index("sitzung")
sitzung_sentiws = sitzung_sentiws_syrien_df.join(sitzung_sentiws_europa_df)

In [ ]:
seaborn.set(style="white", rc={'figure.figsize':(12,6)})
plot = seaborn.lineplot(data=sitzung_sentiws)
plt.setp(plot.set_xticklabels([]))